In [ ]:
import sys
import os
import numpy as np 
import matplotlib.pyplot as plt

src_path = os.path.expanduser('~/source/')
sys.path.append(src_path)

In [ ]:
#greg tools
sys.path.append(src_path+'simtools')
sys.path.append(src_path+'simtools/infoenginessims')

from integrators import rkdeterm_eulerstoch
from dynamics import langevin_underdamped
import simulation
from simprocedures import basic_simprocedures as sp
from simprocedures import running_measurements as rp
from simprocedures import trajectory_measurements as tp
#from infoenginessims.api import *

In [ ]:
#kyle tools
from sus.protocol_designer import Potential, Protocol, System, Compound_Protocol
from informational_states.measure import MeasurementDevice, Measurement, TrajectoryEnsemble
import kyle_tools as kt

In [ ]:
# SQUID tools
from FQ_sympy_functions import DeviceParams
from bit_flip_sweep import set_systems, check_device

In [ ]:
plt.rcParams['font.size'] = 16
legendsize = 6
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

rc_dict = {'axes.labelsize':'large', 'ytick.right':False,'legend.loc':'upper right', 'legend.fontsize':'xx-small', 'figure.autolayout':True, 'figure.figsize': (12,9)}
plt.rc('grid', linestyle="-", color='black')

for key,value in rc_dict.items():
    plt.rcParams[key] = value

# These are the physical parameters that characterize the system

In [ ]:
# set up the device here, for example:
Dev = DeviceParams()
_PHI_0 = 2.067e-15

## doing changes in this order incase overwrites don't work
print('Before change')
print('-------------')
print(f'I_plus, I_minus: {Dev.I_plus}, {Dev.I_minus}')
print(f'L, ell: {Dev.L}, {Dev.ell}')
print(f'R, C: {Dev.R}, {Dev.C}')
print('--------------')

Dev.change_vals({'I_plus':20e-6, 
                 'I_minus':0, 
                 'L':26.3e-12, 
                 'ell':6.58e-12,
                 'R':1_000, 
                 'C':0.03e-12, # from T2018 cited process
                  })

print('After change 1: ')
print('-------------')
print(f'I_plus, I_minus: {Dev.I_plus}, {Dev.I_minus}')
print(f'L, ell: {Dev.L}, {Dev.ell}')
print(f'R, C: {Dev.R}, {Dev.C}')
print('--------------')
print('--------------')

print('Before change 2:')
print('----------------')
print(f'U_0: {Dev.U_0}')
print(f'gamma: {Dev.gamma}')
print(f'beta: {Dev.beta}')
print('----------------')

# changing these according to Change 1
Dev.change_vals({'U_0':_PHI_0**2/(4*np.pi**2*Dev.L),
                 'gamma':Dev.L / (2*Dev.ell),
                 'beta':2*np.pi*Dev.I_plus*Dev.L/_PHI_0
                  })

print('After change 2:')
print('----------------')
print(f'U_0: {Dev.U_0}')
print(f'gamma: {Dev.gamma}')
print(f'beta: {Dev.beta}')
print('----------------')
print('----------------')

check_device(Dev)
print('Before change 3')
print('----------------')
print(f'kT_prime (k_BT/U_0): {Dev.kT_prime}')
print('----------------')

# For R2023 FQ
#Dev.change_vals({'kT_prime':((0.85*1.38e-23)/Dev.U_0)})
# For T2018 QFP
Dev.change_vals({'kT_prime':((4.2*1.38e-23)/Dev.U_0)})

print('After change 3')
print('----------------')
print(f'kT_prime (k_BT/U_0): {4.2*1.38e-23/Dev.U_0}')
print('----------------')

In [ ]:
check_device(Dev)

In [ ]:
#params Dev:
kT = Dev.get_temp()*1.38E-23
C = Dev.C
R = Dev.R
L = Dev.L

#these are some relevant dimensionful scales: alpha is the natural units for the JJ fluxes and U_0 is the natural scale for the potential
alpha = Dev.alpha

#IMPORTANT: all energies are measured in units of U_0
U_0 = alpha**2 / L
h = 6.63E-34

#these are important dimensionless simulation quantities, accounting for 
#m being measured in units of C, lambda in units of 1/R, energy in units of U_0
m_prime = np.array((1, 1/4))
lambda_prime = np.array((2, 1/2))
kT_prime = Dev.kT_prime
U_0 = Dev.U_0

print('Common Circuit Parameters characterizations')
print('------------')
print(f'characteristic time sqrt(LC):     {np.sqrt(L*C):.2e}')
Q = R*np.sqrt(C/L)
print( 'Q:                                {:.2f}'.format(Q))
frq = 1/(2*np.pi*np.sqrt(L*C))
print('f_LC in GHz:                      {:.2f}'.format(frq/1E9))
print('ring down:                        {:.2f} ns'.format(1E9*Q/frq))
print('ring down in sqrt(LC):            {:.2f}'.format( (Q/frq)/np.sqrt(L*C)))
j_c = alpha/L
print('I_c density (microAmps):          {:.2f}'.format(1E6*j_c))
print('Energy Scale ~ U_0/kT:            {:.2f}'.format(1/Dev.kT_prime))
print(f'quantumness if < 1 ~ kT/(h*f):    {Dev.kT_prime*U_0 / (h*frq):.2f}',)

In [ ]:
print('Some device checks')
print('------------------')
print(f'Dev.alpha, Phi_0/(2pi): {Dev.alpha:.2e}, {_PHI_0/(2*np.pi):.2e}')
print(f'alpha**2/L, Phi_0**2/4pi**2 L: {Dev.alpha**2/Dev.L:.2e}, {_PHI_0**2/(4*np.pi**2*Dev.L):.2e}')
print('------------------')
Dev.get_params()
print('------------------')
print(f'device temp: {Dev.get_temp()}')

# Here we define the $\varphi_{xdc}$ protocol

In [ ]:
### protocol 1: matching protocol to how I'd expect ###
# this is the "flux qubit" potential, it depends on 5 parameters: [phi_x, phi_xdc, gamma, Beta, deltaBeta]
from sus.library.fq_systems import fq_pot
import sus.protocol_designer as designer

# define the default protocol values, we will only change phi_xdc so the others will stay at the default values.
default_values = [0, 0, Dev.gamma, Dev.beta, Dev.dbeta]
print(f'default_values: {default_values}')

# changing the first and second parameters
which_params = [1, 2]

# T2018 phi_+ coordinate is a typo
# changing phi_xdc parameters accordingly
double_well = -0.01
single_well = -6.28
#single_well = -3.14

# phi_x parameters
no_tilt = 0
tilt = 0.24

# when protocol begins
start_time = 5

# rise time  = sqrt(LC) \approx 1 ns like in Takeuchi 2018
#tau_rise = 1125 
tau_rise = 788 # 70% of previous tau_rise, so is a faster protocol just like bitswap

'''  Takeuchi 2018 steps
 1) initialization
 -------- skipping 1 ns pause for now
 2) barrier drop start
 3) barrier drop end 
 --------  skipping 1 ns pause for now
 4) tilt start
 5) tilt end
 -------- skipping 1 ns pause for now
 6) raise barrier start
 7) raise barrier end
 -------- skipping 1 ns pause for now
 8) untilt start
 9) untilt end
 -------- skipping 1 ns pause for now
'''
protocol_steps_dict = {
    '1: init': {'pxdc': double_well, 'px': no_tilt, 'time': 0},
    '2: b-drop beg': {'pxdc': double_well, 'px': no_tilt, 'time': start_time},
    '3: b-drop end': {'pxdc': single_well, 'px': no_tilt, 'time': start_time + tau_rise},
    '4: tilt beg': {'pxdc': single_well, 'px': no_tilt, 'time': start_time + tau_rise},
    '5: tilt end': {'pxdc': single_well, 'px': tilt, 'time': start_time + 2*tau_rise},
    '6: b-raise beg': {'pxdc': single_well, 'px': tilt, 'time': start_time + 2*tau_rise},
    '7: b-raise end': {'pxdc': double_well, 'px': tilt, 'time': start_time + 3*tau_rise},
    '8: untilt beg': {'pxdc': double_well, 'px': tilt, 'time': start_time + 3*tau_rise},
    '9: untilt end': {'pxdc': double_well, 'px': no_tilt, 'time': start_time + 4*tau_rise}
}

# set up lists of values for protocol
phi_xdc_values, phi_x_values, keyframe_times = [], [], []

for protocol_step, step_info_dict in protocol_steps_dict.items():
    phi_xdc_values.append(step_info_dict['pxdc'])
    phi_x_values.append(step_info_dict['px'])
    keyframe_times.append(step_info_dict['time'])

phi_values = [phi_x_values, phi_xdc_values]

#protcol has 5 parameters to keep track of
num_params = 5
# and the number of steps is the number of times minus one
num_steps = len(keyframe_times)-1

# this function allows us to set a time dependent potential over several steps
keyframe_protocol = designer.protocol.sequential_protocol

# now use the keyframe_protocol function to create the protocol
protocol = keyframe_protocol(num_steps, num_params, which_params, phi_values, keyframe_times, default_values)

# after the fact, you can go through and change the interpolation method for each step. Uf you set this boolean to "true" it will do instant steps instead of linear
# this will mean there is no ramp up time.
use_step_interpolation = False
if use_step_interpolation:
    protocol.protocols[1].interpolation = 'step'
    protocol.protocols[-1].interpolation = 'end_step'

# now we define the system, a class that applies the created protocol to the potential for the flux qubit
comp = System(protocol, fq_pot)

In [ ]:
# plots the protocol parameters over time, time is in units of sqrt(LC)
fig, ax = comp.protocol.show_params(which=[1,2], resolution=10_000, param_labels=['$\\varphi_{x}$', '$\\varphi_{xdc}$'], show=False);
fig.set_size_inches(10, 4)

In [ ]:
#this cell defines what system you want to simulate and how many trials to run.
#generally no need to do lots of trials while prototyping protocols

N = 5_000
system = comp
eq_sys = comp

system.mass= m_prime
eq_sys.mass = m_prime

system.potential.scale=1

#initialize the state in an EQ distribution.

### SETTING THE INITIAL TIME DOESN'T WORK --- NEED TO FOLLOW UP!
# why doesn't this happen in the "auto" protocol for [0.3, 0.3],
# but this is happening right now?
# changing below
#initial_state = eq_sys.eq_state(N, t=0, beta=1/(kT_prime), resolution=1_000, manual_domain=[[-4,-4],[4,0]], axes=[1,2])
initial_state = eq_sys.eq_state(N, t=0, beta=1/(kT_prime), resolution=1_000, manual_domain=[[-4,-2],[4,2]], axes=[1,2])

# Next few cells are visualization checks that your system is set up how you want

In [ ]:
#this cell checks for closeness to equilibrium by checking the generalized equipartition theorem
#the true equilibrium dist will yield zeros for all elements, but only when when n -> infinity
#this can take a lot of trials to converge

I = np.zeros((4,4))
for i in range(4):
    I[i,i] = 1

ept_test = system.check_local_eq(initial_state, 0)/kT_prime - I 
ept_mean, ept_std = np.mean(ept_test, axis=0), np.std(ept_test, axis=0)

element_labels = [ f'${i+1},{j+1}$' for i in range(4) for j in range(4)]

fig, ax = plt.subplots(figsize=(14,4))

ax.errorbar(element_labels, ept_mean.ravel(), yerr = 3*ept_std.ravel()/np.sqrt(N), fmt='o', capsize=3, capthick=2, elinewidth=2, color='black')
ax.axhline(linestyle='--')
ax.set_xlabel('$i,j$')
fig.suptitle('Generalized Equipartition Theorem: '+'$\\langle x_i \\partial_{x_j} H\\rangle - I \\text{ with } x_1, x_2, x_3, x_4 = \\varphi, \\varphi_{dc}, \\dot{\\varphi}, \\dot{\\varphi}_{dc}$')

In [ ]:
#this cell is for checking that the initial state is what you want qualitatively, shows phase space histograms
nbins= 30

fig, ax = plt.subplots(2,2, figsize=(10,10))

for i in range(0, 1+1, 1):
    for j in range(0, 1+1, 1):
        ax[i, j].hist(initial_state[:, i,j], bins=nbins, density=True)

titles = ['$\\phi$', '$\\dot{\\phi}$', '$\\phi_{dc}$', '$\\dot{\\phi}_{dc}$']
for i, axis in enumerate(ax.ravel()):
    axis.set_title(titles[i])

In [ ]:
system.protocol

In [ ]:
#gives a snapshot of the potential at some time in some domain
fig, ax = plt.subplots(1,2, figsize=(12,4))
t_snapshot_init = system.protocol.t_i 

# for quasistatic erasure
_domain_i = [[-2.25, -0.5],[2.25, 0.5]]

fig, _, out = system.show_potential(t_snapshot_init, manual_domain=_domain_i, figax=[fig,ax[0]], surface=False, show=False)
print(f't_i, t_f: {system.protocol.t_i}, {system.protocol.t_f}')

t_snapshot_final = system.protocol.t_i + 2*tau_rise
_domain_f = [[-2.5,-2.5],[2.5, -4]]
fig, ax[1], out = system.show_potential(t_snapshot_final, manual_domain=_domain_f, figax=[fig,ax[1]], surface=False, show=False)

# Now we set up the simulation

In [ ]:
# this sets up our simulation to do underdamped langevin dynamics.
# if you want to change the temperature or damping by some amount, you can change the scale factors in this cell
# probably dont want to change anythign else in here though

#NOTE: changing the temperature here will not change the temperature used to generate the EQ distribution,
#NOTE: time is scaled in units of sqrt(LC)

gamma = (lambda_prime/m_prime) * np.sqrt(L*C) / (R*C) 
theta = 1/m_prime
eta = (L/(R**2 * C))**(1/4) * np.sqrt(kT_prime*lambda_prime) / m_prime        

damping_scale = 1
temp_scale = 1

gamma = np.multiply(gamma, damping_scale)
eta = np.multiply(eta, np.sqrt(damping_scale*temp_scale))

dynamic = langevin_underdamped.LangevinUnderdamped(theta, gamma, eta, system.get_external_force)
dynamic.mass = system.mass

integrator = rkdeterm_eulerstoch.RKDetermEulerStoch(dynamic)

In [ ]:
#dont change this cell unless you take a look at how the procedures work, this should be fine for most use cases

is_bools = kt.separate_by_state(initial_state[...,0,0])

## commenting out certain procedures for memory saving
# all work is for trajectory information
# final work is keeping track of work moment to moment, and just keeps array
procedures = [
              sp.ReturnFinalState(),
              sp.MeasureAllState(trial_request=slice(0, 1000), step_request=np.s_[::4]),
              #rp.MeasureAllValue(rp.get_dW, 'all_W'),
              rp.MeasureFinalValue(rp.get_dW, 'final_W'),
              #sp.MeasureMeanValue(rp.get_kinetic, output_name='kinetic' ),
              #sp.MeasureMeanValue(rp.get_potential, output_name='potential'),
              #sp.MeasureMeanValue(rp.get_EPT, output_name='equipartition'),
              sp.MeasureMeanValue(rp.get_current_state, output_name = 'zero_means', trial_request=is_bools['0']),
              sp.MeasureMeanValue(rp.get_current_state, output_name = 'one_means', trial_request=is_bools['1']),
              tp.CountJumps(state_slice=np.s_[...,0,0])
             ]

In [ ]:
# here is where you choose the number of steps to simulate and how long to run the sim for.
# note that if your time is longer than the protocol time, the potential will just sit at its final value.
print(f'{system.protocol.t_f-system.protocol.t_i}')
total_time = (system.protocol.t_f-system.protocol.t_i) + 2

#dt = 0.2e-12 / np.sqrt(Dev.L*Dev.C) for Takeuchi 2018
#   = 0.23
#dt = 1
dt = 0.05 # making super small
#print(f'dt: {dt}')
#print(f'tau_rise (\sqrt(LC)) = 1e-9 / dt = {1e-9 / (dt*np.sqrt(Dev.L*Dev.C))}')
#print(f'tau_rise for verification: {tau_rise}') # could throw try-except error but not doing it for now
#print(f'===> go back and change if not the same')
print(f'total time: {total_time*np.sqrt(Dev.L * Dev.C):.2e}:')
nsteps = int(total_time/dt)
nsteps_quick = int(nsteps/10)

sim = simulation.Simulation(integrator.update_state, procedures, nsteps, dt,
                            initial_state)

sim.system = system

# Run simulation

In [ ]:
%%time
sim.output = sim.run(verbose=True)

In [ ]:
# Here we assign sim output to some variables for easier access later
# this is a bit of a mess, but it works for now

# this saves trajectory data for the first 1,000 samples. It only saves every 4 time steps to save on memory. this is set in procedures
all_state = sim.output.all_state['states']
# work over time, all trials, all time steps
#all_W = sim.output.all_W
# net work at the end per trial
final_W = sim.output.final_W
# the full final state
final_state = sim.output.final_state
# equipartition checks over time, ['values'] is the mean, ['std_error'] is the std error
#all_EPT = sim.output.equipartition['values']
# average KE, PE of all particles over time
#all_KE = sim.output.kinetic['values']
#all_PE = sim.output.potential['values']
# arrays to plot time on the x axis
times = np.linspace(0, total_time, nsteps+1)
# coarse grained version for plotting all_state
all_times = np.linspace(0, total_time, all_state.shape[1])
# info state conditioned averages for each phase space coordinate
z_states = sim.output.zero_means['values']
z_err = np.sqrt(N)*sim.output.zero_means['std_error']
o_states = sim.output.one_means['values']
o_err = np.sqrt(N)*sim.output.one_means['std_error']

# this measures every time the sign of phi changes form + to - 
jumps = sim.output.trajectories

In [ ]:
import json
fig, ax = plt.subplots(1,1, figsize=(8,6))

label_size = 20
plt.rc('xtick', labelsize=label_size)
plt.rc('ytick', labelsize=label_size)
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'

#ax.set_title('Final Work (kT) histogram')
final_W_kT = final_W/kT_prime
ax.hist(final_W_kT, bins=200, color='darkorange', density=True)
m=(final_W_kT).mean()
s=(final_W_kT).std()
ax.axvline(m, color='k', label=f'Mean = {m:.2f}')
ax.axvline(m-3*s, color='r')
ax.axvline(m+3*s, color='r')

ax.set_xlabel(r'$W / k_BT$', fontsize=30)
ax.set_ylabel(r'$\mathrm{counts}$', fontsize=30)

ax.legend(fontsize=15)

print(f'mean: {m}')
#with open('Langevin_work_erasure-list-for-5000-sims-shorter-dynamics', 'w') as f:
#    json.dump(final_W_kT.tolist(), f)

In [ ]:
####### WARNING: This plot takes up a LOT of RAM.
fig, ax = plt.subplots()
all_ke  = system.get_kinetic_energy(all_state) / kT_prime

ax.plot(all_times, all_ke.T, alpha=.005);
ax.plot(times, all_KE/kT_prime, zorder=10000, linewidth=1, color='k', label='ensemble average' );
ax.axhline(1, linestyle='--', color='k', alpha=.2, label='kT')
fig.legend()
fig.suptitle('total Kinetic Energy')

In [ ]:
fig, ax = plt.subplots(2,2)

coarse = 25
for i in range(2):
    for j in range(2):
        for state, error in zip([z_states, o_states],[z_err, o_err]):
            markers, caps, bars = ax[i,j].errorbar(times[::coarse], state[::coarse,i,j], yerr=error[::coarse,i,j], alpha=.6);
            [bar.set_alpha(.2) for bar in bars]
            for lineval in [ state[0,i,j] + item*error[0,i,j] for item in[-1,1]]:
                ax[i,j].axhline(lineval, linestyle='--', color='k', alpha=.2)

_ax_fontsize = 25
ax[0,0].set_ylabel('$\\varphi$', fontsize=_ax_fontsize)
ax[0,1].set_ylabel('$\\dot{\\varphi}$', fontsize=_ax_fontsize)
ax[1,0].set_ylabel('$\\varphi_{\mathrm{dc}}$', fontsize=_ax_fontsize)
ax[1,1].set_ylabel('$\\dot{\\varphi}_{\mathrm{dc}}$', fontsize=_ax_fontsize)

#ax[0,0].set_xticklabels([])
ax[0,1].set_xticklabels([])
ax[0,0].set_xlabel(r'$t \; (\sqrt{LC})$', fontsize=_ax_fontsize)

ax[1,0].set_xlabel(r'$t \; (\sqrt{LC})$', fontsize=_ax_fontsize)
ax[1,1].set_xlabel(r'$t \; (\sqrt{LC})$', fontsize=_ax_fontsize)

xadjust = 2
#endtime = 5+(2*ramp_time)
#for i in range(0, 1+1, 1):
#    for j in range(0, 1+1, 1):
#        ax[i,j].axvline(x=5, linestyle='--', color='black')
#        ax[i,j].axvline(x=endtime, linestyle='--', color='black')
#       ax[i,j].set_xlim([5-xadjust, endtime+xadjust])
        
#ax[0,0].set_title('$\\phi$')
#ax[0,1].set_title('$\\dot{\\phi}$')
#ax[1,0].set_title('$\\phi_{dc}$')
#ax[1,1].set_title('$\\dot{\\phi}_{dc}$')

#fig.suptitle('Info State Average and Std Deviation')

In [ ]:
fig, ax = plt.subplots(2,2)

coarse = 25
for i in range(2):
    for j in range(2):
        for state, error in zip([z_states, o_states],[z_err, o_err]):
            markers, caps, bars = ax[i,j].errorbar(times[::coarse], state[::coarse,i,j], yerr=error[::coarse,i,j], alpha=.6);
            [bar.set_alpha(.2) for bar in bars]
            for lineval in [ state[0,i,j] + item*error[0,i,j] for item in[-1,1]]:
                ax[i,j].axhline(lineval, linestyle='--', color='k', alpha=.2)


ax[0,0].set_title('$\\phi$')
ax[0,1].set_title('$\\dot{\\phi}$')
ax[1,0].set_title('$\\phi_{dc}$')
ax[1,1].set_title('$\\dot{\\phi}_{dc}$')

fig.suptitle('Info State Average and Std Deviation')

In [ ]:
'''
fig, ax = plt.subplots(1,1, figsize=(15,5))

#ax[0].set_title('Work (kT) vs time')
#all_W_kT = all_W/kT_prime
#all_W_kT = all_W

#ax[0].plot(times, all_W_kT.T, alpha=.1);
#ax[0].plot(times, all_W_kT.mean(axis=0), linewidth=2, color='k', label='ensemble average', alpha=1 );

#ax[0].legend()

#this will show you a histogram of the net work, with the mean and +- sigma marked
#note the energy scale is in k_B T

ax[1].set_title('Final Work (kT) histogram')
final_W_kT = final_W/kT_prime
ax[1].hist(final_W_kT, bins=30)
m=(final_W_kT).mean()
s=(final_W_kT).std()
ax[1].axvline(m, color='k', label='mean')
ax[1].axvline(m-3*s, color='k', label=' - 3 sigma')
ax[1].axvline(m+3*s, color='k', label=' + 3 sigma')
ax[1].legend()

print(m)
'''

In [ ]:
for item in rc_dict:
    plt.rcParams[item] = rc_dict[item]

# dont need ot look at all of them, this will show every 10th
fig, ax = plt.subplots(2,2, sharex=True)

opacity_values = [.5, .01]
trial_indices = np.s_[::1], np.s_[::1]

#for i, [opacity, trials] in enumerate(zip(opacity_values, trial_indices)):
#    for j, xy in enumerate([[0,0],[1,0]]):
#        ax[i,j].plot(all_times, all_state[trials,:,*xy].T, alpha=opacity);

# have to set up the indices this way
for j, [opacity, trials] in enumerate(zip(opacity_values, trial_indices)):
    for i in range(0, 1+1, 1):
        ax[i,j].plot(all_times, all_state[trials,:, i,j].T, alpha=opacity);
        
fig.suptitle('all_state trajectories with t in units of $\\sqrt{LC}$')

ax[0,0].set_ylabel('$\\varphi$')
ax[0,1].set_ylabel('$\\varphi_{dc}$')
ax[1,0].set_ylabel('$\\varphi$')
ax[1,1].set_ylabel('$\\varphi_{dc}$')

tick_params = {'which':'major', 'labelsize':12, 'size':2, 'direction':'inout', 'width':.6}
for item in ax.ravel():
    item.axvline(system.protocol.t_f,c='k', linestyle='dashed')
    item.axvline(1,c='k', linestyle='dashed')
    #item.axvline(5.53,c='k', linestyle='dashed')
    item.grid(True, which='both')
    item.tick_params(**tick_params)

In [ ]:
#fig.savefig('all_state_alpha.pdf')

In [ ]:
from FQ_sympy_functions import fidelity
fidelity_dictionary = fidelity(jumps)
print(fidelity_dictionary)

# Below This are more detailed Diagnostic Tools for the protocol and Simulation dt

In [ ]:
from bit_flip_sweep import get_tau_candidate

# this will take a look at this sim and suggest candidate end times for the protocol
# it makes a guess of a better time based only on the phi coordiante
# but its guess is usually wrong because phi_dc matters a lot
tau_candidates, t_crit = get_tau_candidate(sim, burn = int( (1+system.protocol.times[0,1]) / sim.dt) )

In [ ]:
# this cell shows a closeup of the flip part, and all the phase space coordinates
# can try to use it to see if extending, shortening will give better results
fig, ax = plt.subplots(2, figsize=(20,10))

#change plot window
#start_idx, end_idx = [ int(item/sim.dt) for item in system.protocol.times[1] ]

## changing for manual protocol since it's not clearnly made like the automatic
start_idx, end_idx = [ int(item/sim.dt) for item in [system.protocol.times[1][0], system.protocol.times[3][1]]]

print(f'protocol times: {system.protocol.times[0]}, {system.protocol.times[3]}')

print(f'start, end: {start_idx}, {end_idx}')
print('\n\n')

duration_idx = int( (end_idx-start_idx)/2)

indices = np.s_[start_idx-duration_idx:end_idx+duration_idx]

for i in range(2):
    for j in range(2):
        for state, error in zip([z_states, o_states],[z_err, o_err]):
            markers, caps, bars = ax[i].errorbar(times[indices], state[indices,i,j], yerr=error[indices,i,j], alpha=1, );
            [bar.set_alpha(.2) for bar in bars]


ax[0].set_title('$\\phi$')

ax[1].set_title('$\\phi_{dc}$')

for item in ax:
    item.grid(True, which='both')
    item.axvline(system.protocol.t_f, linewidth=5, color='r',alpha=.4, label='end of protocol')
    item.axvline(tau_candidates[0], color='b', linestyle='--', label='range_start')
    item.axvline(tau_candidates[1], color='b', linestyle='--', label='range_end',)
    item.axvline(t_crit, linewidth=5, color='g', alpha=.4, label='guessed better time')
    #item.axvline(18.18, linewidth=5, color='orange')
    
    
print(f'tau_candidates: {tau_candidates[0]}, {tau_candidates[1]}')
print(f'system.protocol.t_i: {system.protocol.t_i}')
print(f'system.protocol.t_f: {system.protocol.t_f}')
print(f'len of protocol (s): {(system.protocol.t_f - 15)*1.36e-9}')
print(f't_crit: {t_crit}')
print(f't_crit - 15 - : {(t_crit - 15)*1.36e-9}')    


ax[1].legend()
fig.suptitle('Ensemble Average of the System Phase Space, conditioned on the initial logical state')

In [ ]:
# looking at energy over time
times = np.linspace(0, total_time, nsteps+1)

fig, ax = plt.subplots(figsize=(15,5));

#potential energy is tricky, because it only makes sense up to a constant-- so i have basically scaled it to look right.
fp_PE = 1.2336799130862295

ax.plot(times, (all_PE - fp_PE)/kT_prime, alpha=.8, label='potential');
#ax.plot(times, (all_PE)/kT_prime, alpha=.8, label='potential');
ax.plot(times, all_KE/kT_prime, alpha=.8, label='kinetic');

ax.legend()
ax.set_title('Avg Energy (k_B T) vs time');
plt.grid(True, which='both')
plt.rc('grid', linestyle="-", color='black')

#ax.set_ylim(0.75, 1.25)
#ax.set_ylim(15,25)

In [ ]:
fig, ax = plt.subplots()
total_E = ( (all_PE-all_PE[0]) + all_KE)/kT_prime +1
ax.plot(times,total_E)
ax.axhline(2, linestyle='dotted')
ax.set_title('Total Energy (k_B T)');
ax.set_ylim([1.5, 2.5])

In [ ]:
# plot equipartition check, this is mostly for checking if the distribution is equilibrium

# dont think of these as energies, moreso it measures correlation
EPT = np.abs(all_EPT/kT_prime)

# we convolve over a time window to smooth out the data
import scipy.signal
window_size = int(2.5/dt)
kernel_1d = np.ones(window_size) / float(window_size)
kernel_3d = kernel_1d.reshape(-1, 1, 1) 
EPT = scipy.signal.convolve(EPT, kernel_3d, mode='valid', method='auto')

fig, ax = plt.subplots(1,2, figsize=(15,5));

for i in range(4):
    for j in range(4):
        if i==j:
            ax[0].plot(EPT[:,i,j]-1, alpha=.8);
        else:
            ax[1].plot(EPT[:,i,j], alpha=.8);

ax[0].set_title(' \'Equipartition Elements\' (k_B T)  (diagonal elements)');
ax[1].set_title(' \'Equipartition Elements\' (k_B T)  (off-diagonal elements)');

ax[0].legend(['phi', 'phi_dc','v_phi','v_phi_dc'])

#manually set window here
#ax[0].set_ylim(-.5,6)
#ax[1].set_ylim(-.5,6)
for i in range(0, 1+1, 1):
    ax[i].set_ylim(-.5, 1)